<img src="images/cas.png" style="width: 300px;"/>


# Data Modeling with Cassandra

### Project Overview

A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.

They'd like a data engineer to create an Apache Cassandra database which can create queries on song play data to answer the questions, and wish to bring you on the project. Your role is to create a database for this analysis. You'll be able to test your database by running queries given to you by the analytics team from Sparkify to create the results.


In this project, data modeling with Apache Cassandra and ETL pipeline using Python is applied. Data is modeled by creating tables in Apache Cassandra to run queries. The implementation includes ETL pipeline that transfers data from a set of CSV files within a directory to create a streamlined CSV file to model and insert data into Apache Cassandra tables.


# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra project

 The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_music 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify_music')
except Exception as e:
    print(e)

## QUERY 1
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

<b>Query Description:</b>
In this query, I used `sessionId` as the partition key and `itemInSession` as my clustering key. Each partition is uniquely identified by sessionId while itemInSession was used to uniquely identify the rows within a partition to sort the data by the value of 4 .  

#### Q1: CREATE TABLE

In [8]:
query = "CREATE TABLE IF NOT EXISTS music_listened_history"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)          

#### Q1: SET UP CSV file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
        query = "INSERT INTO music_listened_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Q1: SELECT STATEMENT

In [10]:
query = "SELECT artist, song, length  from music_listened_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
   
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875



## QUERY 2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

<b>Query Description:</b>
In this query, I used `userId` and `sessionId` as my composite partition keys and `itemInSession` as my clustering key. Each partition is uniquely identified by the composite partition keys and itemInSession was used to uniquely identify the rows within partitions to sort the data. From there the clustering column will sort in order of how they were added to the primary key and in this case rows are ordered by the itemInSession column.

#### Q2: CREATE TABLE

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_user_session"
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

#### Q2: SET UP CSV file

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
        query = "INSERT INTO  song_user_session (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Q2: SELECT STATEMENT

In [13]:
query = "SELECT artist, song, firstname, lastname from song_user_session WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song , row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## QUERY 3
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


<b>Query Description:</b>
In this query, I used `song` as my partition key and `userId` as the clustering key. The keys song, and userId are combined to form a unique hash to identify each record (row). This is done since many users could have the same first name and last name and might have listened to the same song (i.e song = 'All Hands Against His Own').


#### Q3: CREATE TABLE

In [14]:
query = "CREATE TABLE IF NOT EXISTS listened_history"
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Q3: SET UP CSV file

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
        query = "INSERT INTO listened_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"    
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Q3: SELECT STATEMENT

In [16]:
query = "SELECT firstname, lastname from listened_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop table: music_listened_history
query = "DROP table music_listened_history"
try:
    rows = session.execute(query)
except Exception as e:  
    print(e)
    
# Drop table: song_user_session
query = "DROP table song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Drop table: listened_history
query = "DROP table listened_history"
try:
    rows = session.execute(query)
except Exception as e:
    
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()